In [1]:
import glob
import re
import pathlib
import json

In [12]:
def read_metadata_from_safetensors(filename):


    with open(filename, mode="rb") as file:
        metadata_len = file.read(8)
        metadata_len = int.from_bytes(metadata_len, "little")
        json_start = file.read(2)

        assert metadata_len > 2 and json_start in (b'{"', b"{'"), f"{filename} is not a safetensors file"

        res = {}

        try:
            json_data = json_start + file.read(metadata_len-2)
            json_obj = json.loads(json_data)
            for k, v in json_obj.get("__metadata__", {}).items():
                res[k] = v
                if isinstance(v, str) and v[0:1] == '{':
                    try:
                        res[k] = json.loads(v)
                    except Exception:
                        pass
        except Exception:
             Exception(f"Error reading metadata from file: {filename}")

        return res

In [2]:
files = glob.glob('../wildcards/**/*.txt', recursive=True) + glob.glob('../wildcards/**/*.yaml', recursive=True)

pattern = re.compile('<lora:(.+?):')

In [11]:
matches = []
for file in files:
    with open(file, 'r') as f:
        matches += pattern.findall(f.read())
matches = set(matches)

In [ ]:
loras = glob.glob('../../../models/Lora/*.safetensors')

ss_names = []

lora_dict = {}

for lora in loras:
    fname = pathlib.Path(lora).stem
    try:
        ss_name = read_metadata_from_safetensors(lora)['ss_output_name']
    except:
        ss_name = fname

    if ss_name in lora_dict.keys():
        lora_dict[ss_name] += [fname]
    else:
        lora_dict[ss_name] = [fname]
    ss_names.append(ss_name)
ss_names = set(ss_names)

In [20]:
matches.difference(ss_names)

{'10_wilykit_il',
 'CHARACTER_Machamp',
 'CHARACTER_Machoke',
 'CorruptPonyXL',
 'CrystalMommy',
 'Dmon ill-000045',
 'Elsa frozen',
 'ExpressiveH_Incase_Mixed_Style_v2_Illustrious_XL_by_UOC',
 'Garuda',
 'HerrscherAGGA2023_Klaxosaur_Princess_Franxx_SDXLPoni_V1',
 'PDXL_artist_tags_v2',
 'Riley_Andersen_A',
 'SerahFarron',
 'Serenica',
 'Shiva',
 'YunaPony',
 'arcane_pony_v2-0-2',
 'bodypaint_ero',
 'dina',
 'dva academy illus-000040',
 'lugia-05',
 'madeline-v1.0',
 'parasite_shoots_pussy',
 'skinbodysuit e3'}